# Performance Table

In [1]:
import pandas as pd
import numpy as np
import os
import sys


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
import os
os.getcwd()
os.chdir('../')


from utils.utils     import *
from utils.constants import *
from utils.metrics   import *

from os import listdir
from os.path import isfile, join

In [3]:
path_ = os.getcwd()

# benchmark

In [4]:
y_train = pd.read_csv(path_ + '\\data' + '\\y_train.csv')

y_test = pd.read_csv(path_ + '\\data' + '\\y_test.csv')

In [5]:
y_predict_average_ini = [1] * len(y_test)

mean_predict = (np.mean(y_train))[0]
y_predict_average = [ x * mean_predict  for x in y_predict_average_ini ]

D:\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [6]:
benchmark_results = pd.concat([y_test,pd.DataFrame({'forecast': y_predict_average})], axis=1)
benchmark_results = benchmark_results.rename(columns={"measurement": "actual"})

In [7]:
benchmark_df = compute_metrics(benchmark_results, EVAL_METRIC_LIST)

In [8]:
benchmark_summary =  (pd.DataFrame(benchmark_df, index = ['benchmark'])*1000).round(3)

In [9]:
benchmark_summary

,rmse,mape,wmape,wbias,wuforec,woforec
benchmark,29.518,32.206,32.35,-3.256,14.547,17.803


# Models Summary

In [10]:
def model_summary(path_, rel_path):
    """
    path_: path for 'water-quality-prediction-08-27-yl-rerun' or similar folder
    
    rel_path: path for saved best models
    
    Returns:
    Two Pd.DataFrame()
    
    performance_summary: eval_metrics summary
    timing_summary:
    
    """
        
    performance_summary = pd.DataFrame()
    timing_summary = pd.DataFrame()

    results_path = path_ + "\\" + rel_path
    results_files = [f for f in listdir(results_path) if isfile(join(results_path, f))]
    filtered_results_files = [word for word in results_files if ".joblib" in word]
    
    for best_model_name in filtered_results_files:
        best_model_file = path_ + "\\" + rel_path + "\\" + best_model_name
        best_model, best_model_info = load_model(best_model_file)
        eval_metrics= best_model_info['eval_metrics']
        eval_metrics = (pd.DataFrame(eval_metrics, index = [best_model_name])*1000).round(3)
        performance_summary = pd.concat([performance_summary, eval_metrics])

        timing = {k:best_model_info[k] for k in ('totalling_fits', 'cv', 'tuning_time',
     'best_model_fitting_time') if k in best_model_info}
        timing['average_tuning'] = timing['tuning_time'] / timing['totalling_fits']

        timing = (pd.DataFrame(timing, index = [best_model_name])).round(3)
        timing_summary = pd.concat([timing_summary, timing])
    return performance_summary, timing_summary
    

# cv5_numerical only

In [11]:
rel_path = 'results\\cv5_numerical only'

In [12]:
performance_summary_numerical_only, timing_summary_numerical_only = model_summary(path_, rel_path)

In [13]:
performance_summary_numerical_only = performance_summary_numerical_only.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\3348829722.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_only = performance_summary_numerical_only.append(benchmark_summary)


In [14]:
performance_summary_numerical_only

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,10.882,9.336,9.476,0.919,5.198,4.278
best_lin_model.joblib,12.035,10.724,10.898,0.409,5.654,5.244
best_mlp_model.joblib,11.619,9.996,10.151,0.114,5.132,5.018
best_rf_model.joblib,11.473,9.711,9.866,0.824,5.345,4.521
best_xgb_model.joblib,10.956,9.227,9.368,0.982,5.175,4.193
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [15]:
print(performance_summary_numerical_only.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  rmse &  mape &  wmape &  wbias &  wuforec &  woforec \\
\midrule
best\_lightGBM\_model.joblib & 10.88 &  9.34 &   9.48 &   0.92 &     5.20 &     4.28 \\
best\_lin\_model.joblib      & 12.04 & 10.72 &  10.90 &   0.41 &     5.65 &     5.24 \\
best\_mlp\_model.joblib      & 11.62 & 10.00 &  10.15 &   0.11 &     5.13 &     5.02 \\
best\_rf\_model.joblib       & 11.47 &  9.71 &   9.87 &   0.82 &     5.34 &     4.52 \\
best\_xgb\_model.joblib      & 10.96 &  9.23 &   9.37 &   0.98 &     5.17 &     4.19 \\
benchmark                  & 29.52 & 32.21 &  32.35 &  -3.26 &    14.55 &    17.80 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\207521426.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(performance_summary_numerical_only.to_latex(float_format = "%.2f"))


In [16]:
timing_summary_numerical_only

,totalling_fits,cv,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,5,152.820,0.188,0.076
best_lin_model.joblib,150,5,8.794,0.079,0.059
best_mlp_model.joblib,40,5,1974.694,218.523,49.367
best_rf_model.joblib,1200,5,1955.061,65.293,1.629
best_xgb_model.joblib,5760,5,2427.986,4.314,0.422


In [17]:
print(timing_summary_numerical_only.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrrr}
\toprule
{} &  totalling\_fits &  cv &  tuning\_time &  best\_model\_fitting\_time &  average\_tuning \\
\midrule
best\_lightGBM\_model.joblib &            2000 &   5 &       152.82 &                     0.19 &            0.08 \\
best\_lin\_model.joblib      &             150 &   5 &         8.79 &                     0.08 &            0.06 \\
best\_mlp\_model.joblib      &              40 &   5 &      1974.69 &                   218.52 &           49.37 \\
best\_rf\_model.joblib       &            1200 &   5 &      1955.06 &                    65.29 &            1.63 \\
best\_xgb\_model.joblib      &            5760 &   5 &      2427.99 &                     4.31 &            0.42 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\3102503486.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(timing_summary_numerical_only.to_latex(float_format = "%.2f"))


# cv5_numerical_only_withstd

In [18]:
rel_path = 'results\\cv5_numerical_only_withstd'

In [19]:
performance_summary_numerical_cv5_numerical_only_withstd, timing_summary_numerical_cv5_numerical_only_withstd = model_summary(path_, rel_path)

In [20]:
performance_summary_numerical_cv5_numerical_only_withstd = performance_summary_numerical_cv5_numerical_only_withstd.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\4054605800.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_cv5_numerical_only_withstd = performance_summary_numerical_cv5_numerical_only_withstd.append(benchmark_summary)


In [21]:
performance_summary_numerical_cv5_numerical_only_withstd

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,10.844,9.327,9.465,0.862,5.164,4.302
best_lin_model.joblib,12.037,10.742,10.916,0.406,5.661,5.255
best_mlp_model.joblib,14.198,12.919,13.161,-5.499,3.831,9.330
best_rf_model.joblib,11.472,9.710,9.865,0.837,5.351,4.514
best_xgb_model.joblib,10.958,9.221,9.363,0.985,5.174,4.189
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [22]:
print(performance_summary_numerical_cv5_numerical_only_withstd.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  rmse &  mape &  wmape &  wbias &  wuforec &  woforec \\
\midrule
best\_lightGBM\_model.joblib & 10.84 &  9.33 &   9.46 &   0.86 &     5.16 &     4.30 \\
best\_lin\_model.joblib      & 12.04 & 10.74 &  10.92 &   0.41 &     5.66 &     5.25 \\
best\_mlp\_model.joblib      & 14.20 & 12.92 &  13.16 &  -5.50 &     3.83 &     9.33 \\
best\_rf\_model.joblib       & 11.47 &  9.71 &   9.87 &   0.84 &     5.35 &     4.51 \\
best\_xgb\_model.joblib      & 10.96 &  9.22 &   9.36 &   0.98 &     5.17 &     4.19 \\
benchmark                  & 29.52 & 32.21 &  32.35 &  -3.26 &    14.55 &    17.80 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\2787992586.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(performance_summary_numerical_cv5_numerical_only_withstd.to_latex(float_format = "%.2f"))


In [23]:
timing_summary_numerical_cv5_numerical_only_withstd

,totalling_fits,cv,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,5,159.920,0.193,0.080
best_lin_model.joblib,150,5,3.323,0.071,0.022
best_mlp_model.joblib,40,5,1948.634,116.050,48.716
best_rf_model.joblib,1200,5,1934.507,64.575,1.612
best_xgb_model.joblib,5760,5,2429.882,4.297,0.422


In [24]:
print(timing_summary_numerical_cv5_numerical_only_withstd.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrrr}
\toprule
{} &  totalling\_fits &  cv &  tuning\_time &  best\_model\_fitting\_time &  average\_tuning \\
\midrule
best\_lightGBM\_model.joblib &            2000 &   5 &       159.92 &                     0.19 &            0.08 \\
best\_lin\_model.joblib      &             150 &   5 &         3.32 &                     0.07 &            0.02 \\
best\_mlp\_model.joblib      &              40 &   5 &      1948.63 &                   116.05 &           48.72 \\
best\_rf\_model.joblib       &            1200 &   5 &      1934.51 &                    64.58 &            1.61 \\
best\_xgb\_model.joblib      &            5760 &   5 &      2429.88 &                     4.30 &            0.42 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\3845423614.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(timing_summary_numerical_cv5_numerical_only_withstd.to_latex(float_format = "%.2f"))


# cv5_withCategorical_withstd

In [25]:
rel_path = 'results\\cv5_withCategorical_withstd'

In [26]:
performance_summary_numerical_cv5_withCategorical_withstd, timing_summary_numerical_cv5_withCategorical_withstd = model_summary(path_, rel_path)


In [27]:
performance_summary_numerical_cv5_withCategorical_withstd = performance_summary_numerical_cv5_withCategorical_withstd.append(benchmark_summary)

C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\3223384742.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_summary_numerical_cv5_withCategorical_withstd = performance_summary_numerical_cv5_withCategorical_withstd.append(benchmark_summary)


In [28]:
performance_summary_numerical_cv5_withCategorical_withstd

,rmse,mape,wmape,wbias,wuforec,woforec
best_lightGBM_model.joblib,11.040,9.218,9.358,0.947,5.152,4.205
best_lin_model.joblib,11.914,10.792,10.958,1.315,6.136,4.822
best_mlp_model.joblib,14.420,14.541,14.608,3.970,9.289,5.319
best_rf_model.joblib,11.711,9.705,9.874,0.653,5.263,4.611
best_xgb_model.joblib,11.139,9.288,9.435,1.122,5.279,4.157
benchmark,29.518,32.206,32.350,-3.256,14.547,17.803


In [29]:
print(performance_summary_numerical_cv5_withCategorical_withstd.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  rmse &  mape &  wmape &  wbias &  wuforec &  woforec \\
\midrule
best\_lightGBM\_model.joblib & 11.04 &  9.22 &   9.36 &   0.95 &     5.15 &     4.21 \\
best\_lin\_model.joblib      & 11.91 & 10.79 &  10.96 &   1.31 &     6.14 &     4.82 \\
best\_mlp\_model.joblib      & 14.42 & 14.54 &  14.61 &   3.97 &     9.29 &     5.32 \\
best\_rf\_model.joblib       & 11.71 &  9.71 &   9.87 &   0.65 &     5.26 &     4.61 \\
best\_xgb\_model.joblib      & 11.14 &  9.29 &   9.44 &   1.12 &     5.28 &     4.16 \\
benchmark                  & 29.52 & 32.21 &  32.35 &  -3.26 &    14.55 &    17.80 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\1183773100.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(performance_summary_numerical_cv5_withCategorical_withstd.to_latex(float_format = "%.2f"))


In [30]:
timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits'] = timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits'] * 5 

In [31]:
timing_summary_numerical_cv5_withCategorical_withstd['average_tuning'] = timing_summary_numerical_cv5_withCategorical_withstd['tuning_time']/ timing_summary_numerical_cv5_withCategorical_withstd['totalling_fits']

In [33]:
timing_summary_numerical_cv5_withCategorical_withstd.round(3)

,totalling_fits,tuning_time,best_model_fitting_time,average_tuning
best_lightGBM_model.joblib,2000,213.437,0.463,0.107
best_lin_model.joblib,150,44.390,0.825,0.296
best_mlp_model.joblib,40,2147.055,65.814,53.676
best_rf_model.joblib,1200,3633.047,115.665,3.028
best_xgb_model.joblib,5760,13320.127,4.156,2.313


In [34]:
print(timing_summary_numerical_cv5_withCategorical_withstd.to_latex(float_format = "%.2f"))

\begin{tabular}{lrrrr}
\toprule
{} &  totalling\_fits &  tuning\_time &  best\_model\_fitting\_time &  average\_tuning \\
\midrule
best\_lightGBM\_model.joblib &            2000 &       213.44 &                     0.46 &            0.11 \\
best\_lin\_model.joblib      &             150 &        44.39 &                     0.82 &            0.30 \\
best\_mlp\_model.joblib      &              40 &      2147.05 &                    65.81 &           53.68 \\
best\_rf\_model.joblib       &            1200 &      3633.05 &                   115.67 &            3.03 \\
best\_xgb\_model.joblib      &            5760 &     13320.13 &                     4.16 &            2.31 \\
\bottomrule
\end{tabular}



C:\Users\Dennis\AppData\Local\Temp\ipykernel_23536\2353243900.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(timing_summary_numerical_cv5_withCategorical_withstd.to_latex(float_format = "%.2f"))
